<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a0b8669fcfd94916b9b40b92a18f728f83bf2ac06cc2285854529f8f8993eea8
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-19 10:41:53
-------------------
qualified stocks: 88
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.60 K
Current:  1.45 C
-------------------
Today PnL: -22.01 K (-0.15%)
Current PnL: -21.21 L (-13.9%)
CY Booked + Current PnL: -7.25 L (-4.75%)
-------------------
Total profit:  1.73 L
Total loss:  -22.94 L
-------------------
Total Booked + Current PnL: 19.73 L (15.7%)
Total Booked PnL: 40.94 L (32.59%)
Curr Year Booked PnL: 13.96 L (9.65%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.35 C
Est FTT PnL: 90.56 L (62.61%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.11%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,149.44,83.0,M-SC,1.95,163475.0,-11084.0,11116.0,-0.58,-6.35,6.80,0.02,234.0,-1.00,1.13,42.20,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,101.07,50.0,M-SC,3.73,86611.0,-14166.0,14256.0,0.19,-14.06,16.46,0.09,253.0,-0.99,0.60,13.79,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-14.93,49.0,X-MC,3.84,88860.0,-1818.0,16723.0,-0.15,-2.00,18.82,16.44,63.0,-0.11,0.61,17.77,X40,ATH,PHARMA
35,ICICIGI,2252.93,-15.07,56.0,X-MC,7.56,188716.0,14396.0,20815.0,0.17,8.26,11.03,20.19,68.0,0.69,1.30,24.62,X40,ATH,INSURANCE
84,VOLTAS,1530.00,-2.04,53.0,X-MC,3.70,208035.0,16293.0,21469.0,0.20,8.50,10.32,19.69,78.0,0.76,1.44,15.87,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,2275.00,-18.52,59.0,X-LC,3.63,327253.0,14365.0,157311.0,3.36,4.59,48.07,54.87,2.0,0.09,2.26,13.25,X40,BTT,IT
69,SONACOMS,806.63,-31.96,68.0,M-SC,6.76,87972.0,-13187.0,53188.0,2.90,-13.04,60.46,39.54,220.0,-0.25,0.61,24.05,AR,ATH,AUTO
32,HCLTECH,1922.01,11.06,74.0,X-LC,4.34,257496.0,15572.0,44264.0,2.81,6.44,17.19,24.73,7.0,0.35,1.78,22.65,X40,ATH,IT
50,LTIM,7201.88,3.99,65.0,H-LC,11.41,265860.0,19709.0,58223.0,2.64,8.01,21.90,31.66,43.0,0.34,1.84,47.51,X200,ATH,IT
13,BSOFT,831.70,-9.16,60.0,H-SC,8.22,99412.0,-41241.0,112674.0,2.63,-29.32,113.34,50.79,151.0,-0.37,0.69,15.75,XR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,104.33,59.0,H-SC,16.05,138528.0,-79968.0,108163.0,-3.94,-36.60,78.08,12.90,161.0,-0.74,0.96,15.19,OX40N,NTT,FINANCE
58,RAJOOENG,143.10,-48.54,28.0,H-SC,18.21,78000.0,-24500.0,65099.0,-2.24,-23.90,83.46,39.61,135.0,-0.38,0.54,0.00,AR,ATH,MISC
64,SATIN,274.00,-25.10,52.0,H-SC,2.51,232972.0,-43312.0,186517.0,-2.14,-15.68,80.06,51.83,146.0,-0.23,1.61,13.49,XY24,NTT,FINANCE
63,SAMMAANCAP,326.00,19.75,54.0,M-SC,36.60,161010.0,11790.0,132382.0,-2.11,7.90,82.22,96.62,245.0,0.09,1.11,74.25,XY25,NTT,FINANCE
51,MASFIN,398.61,-17.80,52.0,H-SC,7.18,93240.0,-4740.0,26340.0,-2.05,-4.84,28.25,22.05,164.0,-0.18,0.64,35.53,XR,ATH,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,STARHEALTH,761.0,35.57,64.0,H-SC,15.59,279290.0,-1833.0,134702.0,-0.42,-0.65,48.23,47.26,171.0,-0.01,1.93,49.64,XY24,NTT,INSURANCE
86,WIPRO,420.0,-13.79,53.0,M-LC,5.15,151609.0,664.0,108795.0,1.51,0.44,71.76,72.51,99.0,0.01,1.05,6.49,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-20.15,42.0,M-SC,10.13,144775.0,-2869.0,162191.0,-1.39,-1.94,112.03,107.91,216.0,-0.02,1.00,1.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,149.44,83.0,M-SC,1.95,163475.0,-11084.0,11116.0,-0.58,-6.35,6.80,0.02,234.0,-1.00,1.13,42.20,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,101.07,50.0,M-SC,3.73,86611.0,-14166.0,14256.0,0.19,-14.06,16.46,0.09,253.0,-0.99,0.60,13.79,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-66.37,44.0,H-SC,1.99,220698.0,-48969.0,85300.0,-0.71,-18.16,38.65,13.47,158.0,-0.57,1.53,12.22,XY24,NTT,PAINTS
44,JCHAC,2282.00,19986.96,64.0,M-SC,1.44,98112.0,-29693.0,29679.0,-0.02,-23.23,30.25,-0.01,233.0,-1.00,0.68,11.10,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,19.75,54.0,M-SC,36.60,161010.0,11790.0,132382.0,-2.11,7.90,82.22,96.62,245.0,0.09,1.11,74.25,XY25,NTT,FINANCE
86,WIPRO,420.00,-13.79,53.0,M-LC,5.15,151609.0,664.0,108795.0,1.51,0.44,71.76,72.51,99.0,0.01,1.05,6.49,XR,NTT,IT
38,INDIAMART,4810.62,-48.34,47.0,H-SC,4.60,128663.0,5327.0,126296.0,-0.76,4.32,98.16,106.72,139.0,0.04,0.89,28.71,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-48.34,47.0,H-SC,4.60,128663.0,5327.0,126296.0,-0.76,4.32,98.16,106.72,139.0,0.04,0.89,28.71,AR,ATH,MISC
86,WIPRO,420.00,-13.79,53.0,M-LC,5.15,151609.0,664.0,108795.0,1.51,0.44,71.76,72.51,99.0,0.01,1.05,6.49,XR,NTT,IT
25,FINCABLES,1641.55,-20.15,42.0,M-SC,10.13,144775.0,-2869.0,162191.0,-1.39,-1.94,112.03,107.91,216.0,-0.02,1.00,1.00,OX40N,ATH,CABLES
85,WHIRLPOOL,2270.00,-43.00,36.0,M-SC,7.86,111605.0,-3193.0,97230.0,0.51,-2.78,87.12,81.92,236.0,-0.03,0.77,32.12,XR,NTT,DURABLES
87,ZYDUSLIFE,1286.17,-15.41,35.0,H-MC,5.24,205972.0,-3602.0,78269.0,0.39,-1.72,38.00,35.63,119.0,-0.05,1.42,14.67,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TMPV,1065.00,-21.86,22.0,X-LC,0.28,239534.0,-36876.0,458037.0,-1.51,-13.34,191.22,152.37,3.0,-0.08,1.66,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,10.04,24.0,X-SC,10.90,83880.0,-44790.0,90087.0,-0.41,-34.81,107.40,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
26,GILLETTE,11206.78,-19.22,28.0,X-SC,6.11,249330.0,-4566.0,86867.0,-0.39,-1.80,34.84,32.42,85.0,-0.05,1.72,13.03,X40,ATH,FMCG
34,HONAUT,58357.33,-26.46,29.0,X-SC,1.31,104535.0,-23403.0,70540.0,0.04,-18.29,67.48,36.84,90.0,-0.33,0.72,7.02,X40N,ATH,ELECTRICAL
56,QUESS,424.00,-54.27,30.0,X-SC,37.28,45655.0,-19351.0,47203.0,-1.08,-29.77,103.39,42.84,83.0,-0.41,0.32,0.00,XY24,NTT,MISC


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TMPV,1065.00,-21.86,22.0,X-LC,0.28,239534.0,-36876.0,458037.0,-1.51,-13.34,191.22,152.37,3.0,-0.08,1.66,0.00,XY24,NTT,AUTO
43,ITC,452.00,-41.15,42.0,X-LC,0.92,197336.0,-2802.0,23700.0,-0.57,-1.40,12.01,10.44,5.0,-0.12,1.36,4.09,X40,NTT,FMCG
34,HONAUT,58357.33,-26.46,29.0,X-SC,1.31,104535.0,-23403.0,70540.0,0.04,-18.29,67.48,36.84,90.0,-0.33,0.72,7.02,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-21.03,58.0,X-MC,1.46,182378.0,5640.0,49096.0,-0.69,3.19,26.92,30.97,75.0,0.11,1.26,16.15,X40,ATH,INSURANCE
67,SIEMENS,4671.50,-1.82,58.0,H-LC,1.76,159955.0,-26140.0,73627.0,-1.55,-14.05,46.03,25.51,49.0,-0.36,1.11,17.87,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,10.04,24.0,X-SC,10.90,83880.0,-44790.0,90087.0,-0.41,-34.81,107.40,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
56,QUESS,424.00,-54.27,30.0,X-SC,37.28,45655.0,-19351.0,47203.0,-1.08,-29.77,103.39,42.84,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
54,PAGEIND,51605.07,-30.06,33.0,X-MC,8.18,155580.0,-8160.0,50844.0,-0.60,-4.98,32.68,26.07,55.0,-0.16,1.08,0.00,X40,ATH,APPARELS
77,TMPV,1065.00,-21.86,22.0,X-LC,0.28,239534.0,-36876.0,458037.0,-1.51,-13.34,191.22,152.37,3.0,-0.08,1.66,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-19.44,30.0,X-MC,6.92,174119.0,-26387.0,123537.0,-0.38,-13.16,70.95,48.45,64.0,-0.21,1.20,0.41,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-54.27,30.0,X-SC,37.28,45655.0,-19351.0,47203.0,-1.08,-29.77,103.39,42.84,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-42.41,35.0,X-SC,3.10,78584.0,-66576.0,142504.0,-0.81,-45.86,181.34,52.31,91.0,-0.47,0.54,5.19,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,10.04,24.0,X-SC,10.90,83880.0,-44790.0,90087.0,-0.41,-34.81,107.40,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-14.93,49.0,X-MC,3.84,88860.0,-1818.0,16723.0,-0.15,-2.00,18.82,16.44,63.0,-0.11,0.61,17.77,X40,ATH,PHARMA
34,HONAUT,58357.33,-26.46,29.0,X-SC,1.31,104535.0,-23403.0,70540.0,0.04,-18.29,67.48,36.84,90.0,-0.33,0.72,7.02,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.96,-25.57,63.0,X-LC,8.12,295226.0,-50730.0,117441.0,1.74,-14.66,39.78,19.28,1.0,-0.43,2.04,9.14,X40,ATH,IT
41,INFY,2275.00,-18.52,59.0,X-LC,3.63,327253.0,14365.0,157311.0,3.36,4.59,48.07,54.87,2.0,0.09,2.26,13.25,X40,BTT,IT
77,TMPV,1065.00,-21.86,22.0,X-LC,0.28,239534.0,-36876.0,458037.0,-1.51,-13.34,191.22,152.37,3.0,-0.08,1.66,0.00,XY24,NTT,AUTO
83,VBL,671.64,-19.21,44.0,X-LC,5.92,289333.0,-26509.0,139169.0,-0.96,-8.39,48.10,35.67,4.0,-0.19,2.00,4.30,X40N,ATH,FMCG
43,ITC,452.00,-41.15,42.0,X-LC,0.92,197336.0,-2802.0,23700.0,-0.57,-1.40,12.01,10.44,5.0,-0.12,1.36,4.09,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7583.33,61.0,L-SC,18.02,84318.0,-9492.0,86106.0,-1.71,-10.12,102.12,81.67,271.0,-0.11,0.58,63.29,XR,NTT,CERAMICS
69,SONACOMS,806.63,-31.96,68.0,M-SC,6.76,87972.0,-13187.0,53188.0,2.90,-13.04,60.46,39.54,220.0,-0.25,0.61,24.05,AR,ATH,AUTO
51,MASFIN,398.61,-17.80,52.0,H-SC,7.18,93240.0,-4740.0,26340.0,-2.05,-4.84,28.25,22.05,164.0,-0.18,0.64,35.53,XR,ATH,FINANCE
71,SURYODAY,216.00,53.51,44.0,H-SC,12.61,140638.0,-38433.0,79685.0,-0.18,-21.46,56.66,23.03,167.0,-0.48,0.97,39.54,XR,NTT,BANKS
81,VAIBHAVGBL,521.00,64.19,50.0,H-SC,10.78,142354.0,-40421.0,151493.0,-1.52,-22.12,106.42,60.77,159.0,-0.27,0.98,28.20,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,149.44,83.0,M-SC,1.95,163475.0,-11084.0,11116.0,-0.58,-6.35,6.80,0.02,234.0,-1.00,1.13,42.20,OX40N,NTT,PAINTS
13,BSOFT,831.70,-9.16,60.0,H-SC,8.22,99412.0,-41241.0,112674.0,2.63,-29.32,113.34,50.79,151.0,-0.37,0.69,15.75,XR,ATH,IT
5,ASIANPAINT,3460.26,0.35,83.0,X-LC,10.86,255350.0,3582.0,49155.0,-0.15,1.42,19.25,20.95,31.0,0.07,1.77,37.57,X40,ATH,PAINTS
81,VAIBHAVGBL,521.00,64.19,50.0,H-SC,10.78,142354.0,-40421.0,151493.0,-1.52,-22.12,106.42,60.77,159.0,-0.27,0.98,28.20,XR,NTT,JEWELLERY
40,INDUSINDBK,1800.00,-695.06,66.0,L-MC,5.71,48159.0,-29048.0,54439.0,-0.36,-37.62,113.04,32.89,259.0,-0.53,0.33,32.63,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.62
1,25,44.45
2,50,75.16


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.56
MC    28.64
LC    24.78
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.94
X40      20.58
X40N     11.51
XY25     10.12
XR        9.47
AR        9.04
OX40N     7.67
X200      1.84
SR        1.00
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.20
X-MC    22.27
X-LC    19.81
M-SC    13.25
X-SC     7.70
H-MC     4.68
H-LC     2.95
L-SC     1.41
M-MC     1.36
M-LC     1.05
L-LC     0.97
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.59,-4.89,38.81
IT,13.55,-13.63,72.34
FINANCE,10.46,-12.25,65.04
MISC,7.38,-21.70,71.56
PAINTS,6.08,-5.18,20.80
ELECTRICAL,5.61,-11.30,50.76
INSURANCE,4.49,2.80,31.46
PHARMA,3.40,-1.19,35.13
AUTO,3.36,-18.48,132.06


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3282999.0,21
AR,1263119.0,10
XR,1259770.0,13
X40,988277.0,14
X40N,705612.0,9
OX40N,683419.0,10
XY25,471119.0,7
SR,275895.0,2
MH,67596.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3479706.0,25
M-SC,1434659.0,16
X-LC,1203720.0,11
X-MC,1170945.0,16
X-SC,725923.0,8
H-MC,401564.0,3
L-SC,257349.0,3
H-LC,131850.0,2
M-LC,108795.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1182768.0      6
           AR         850401.0      5
           XR         781986.0      7
M-SC       XY24       760760.0      6
X-LC       XY24       528937.0      2
X-MC       X40        419452.0      7
X-LC       X40        391871.0      5
X-MC       XY24       345026.0      3
H-SC       OX40N      321060.0      4
X-SC       XY24       281078.0      3
H-SC       SR         275895.0      2
M-SC       OX40N      274484.0      5
X-SC       X40N       267891.0      3
X-MC       X40N       229673.0      4
H-MC       AR         217134.0      2
X-LC       X40N       208048.0      2
H-MC       XY24       184430.0      1
X-SC       X40        176954.0      2
X-MC       XY25       176794.0      2
L-SC       XR         169474.0      2
M-SC       XR         145076.0      2
           XY25       132382.0      1
           AR         121957.0      2
M-LC       XR         108795.0      1
L-SC       OX40N       87875.0      1
X-LC       XY25        74864.0      2
H-LC       AR          73627.0      1
H-SC       MH          67596.0      1
H-LC       X200        58223.0      1
L-MC       XR          54439.0      1
M-MC       XY25        48287.0      1
L-LC       XY25        38792.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
